In [1]:
import tensorflow as tf
import keras,os,wandb

In [2]:
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
wandb.init(project="image-net-classification-myLaptop")

Take Dataset from Directory

In [3]:
def normalize_img(image, label):
    
    return tf.cast(image, tf.float32) / 255.0, label

In [4]:
BATCH_SIZE = 16

train_ds = keras.utils.image_dataset_from_directory(
    directory="dataset\\train",
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size = (64,64),
    shuffle=True,
    )
val_ds = keras.utils.image_dataset_from_directory(
    directory="dataset\\val",
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size = (64,64),
    shuffle=True,
    )

Found 100000 files belonging to 200 classes.
Found 10000 files belonging to 200 classes.


In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.map(normalize_img,num_parallel_calls=AUTOTUNE).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(normalize_img,num_parallel_calls=AUTOTUNE).cache().prefetch(buffer_size=AUTOTUNE)

In [6]:
image_batch, label_batch = next(iter(train_ds))
image_batch[0],label_batch[0]

(<tf.Tensor: shape=(64, 64, 3), dtype=float32, numpy=
 array([[[0.50980395, 0.49803922, 0.47058824],
         [0.6666667 , 0.654902  , 0.627451  ],
         [0.6431373 , 0.6313726 , 0.6039216 ],
         ...,
         [0.40392157, 0.39607844, 0.4       ],
         [0.29803923, 0.2901961 , 0.29411766],
         [0.7490196 , 0.7411765 , 0.74509805]],
 
        [[0.5686275 , 0.5568628 , 0.5294118 ],
         [0.59607846, 0.58431375, 0.5568628 ],
         [0.54901963, 0.5372549 , 0.50980395],
         ...,
         [0.6039216 , 0.59607846, 0.6       ],
         [0.7058824 , 0.69803923, 0.7019608 ],
         [0.78039217, 0.77254903, 0.7764706 ]],
 
        [[0.59607846, 0.5921569 , 0.57254905],
         [0.6       , 0.59607846, 0.5764706 ],
         [0.6       , 0.59607846, 0.5764706 ],
         ...,
         [0.78431374, 0.7764706 , 0.78039217],
         [0.8       , 0.7921569 , 0.79607844],
         [0.7019608 , 0.69411767, 0.69803923]],
 
        ...,
 
        [[0.50980395, 0.49019608, 

Create Model

In [7]:
from keras.layers import Layer,Conv2D,Dense,BatchNormalization,MaxPooling2D,GlobalAveragePooling2D,Dropout
from keras import Model
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy

In [8]:
class myCNNBlock(Layer):

    def __init__(self,output_channels,kernel_size=3,pool_size=3):

        super(myCNNBlock, self).__init__()

        

        self.conv = Conv2D(output_channels,kernel_size=kernel_size,padding="same")
        self.pooling = MaxPooling2D(pool_size=pool_size)
        self.bn = BatchNormalization()

    def call(self,input_tensor,training=False):

        x = self.conv(input_tensor)
        x = self.bn(x,training=training)
        #x = self.pooling(x)
        x = tf.nn.relu(x)
        return x

class myCNN(Layer):

    def __init__(self,channels):

        super(myCNN, self).__init__()
   
        self.cnn1 = myCNNBlock(output_channels=channels[0])
        self.cnn2 = myCNNBlock(output_channels=channels[1])
        self.cnn3 = myCNNBlock(output_channels=channels[2])
        
        self.maxPool = MaxPooling2D()
        self.identity_mapping = Conv2D(channels[1],3,padding="same")
    
    

    def call(self,input_tensor,training=False):

        x = self.cnn1(input_tensor,training=training)
        x = self.cnn2(x,training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor),training=training)
      
        x = self.maxPool(x)
       

        return x

class fullModel(Model):

    def __init__(self,number_of_classes=200,):

        super(fullModel,self).__init__(name="fullModel")

        self.lightM1 = myCNN([32,32,64])
        self.lightM2 = myCNN([64,64,128])
        self.lightM3 = myCNN([128,256,512])
        self.lightM4 = myCNN([256,512,1024])

        self.pool = GlobalAveragePooling2D()
        self.dropout = Dropout(0.5)
        self.dense1 = Dense(512)
        
       
        self.classifier = Dense(number_of_classes)

    def call(self,input_tensor,training=False):

        x = self.lightM1(input_tensor,training=training)
        x = self.lightM2(x,training=training)
        x = self.lightM3(x,training=training)
        x = self.lightM4(x,training=training)
        x = self.pool(x)
        x = self.dense1(x)
        #x = self.dropout(x)
        x = self.classifier(x)
        
        return x 
    
    def model(self,):
        x = keras.Input(shape = (64,64,3))

        return Model(inputs=[x],outputs = self.call(x))

In [9]:
model = fullModel().model()

model.compile(optimizer = Adam(learning_rate=0.0001),loss=SparseCategoricalCrossentropy(from_logits=True),metrics="accuracy")

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 my_cnn (myCNN)              (None, 32, 32, 64)        30048     
                                                                 
 my_cnn_1 (myCNN)            (None, 16, 16, 128)       185664    
                                                                 
 my_cnn_2 (myCNN)            (None, 8, 8, 512)         1921664   
                                                                 
 my_cnn_3 (myCNN)            (None, 4, 4, 1024)        9446656   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                             

In [11]:
filepath="model/weights-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy',save_weights_only=True, save_best_only=False)

In [12]:
#model.load_weights("model\weights-14-0.17.hdf5")

In [13]:
model.fit(train_ds,validation_data=val_ds,epochs=200,callbacks=[checkpoint],verbose=2)


Epoch 1/200


In [ ]:
import sys
sys.exit(0)

In [ ]:
class myCNNBlock(Layer):

    def __init__(self,output_channels,pool_size=3,kernel_size=3,):
        super(myCNNBlock,self).__init__()

        self.conv = Conv2D(output_channels,kernel_size=kernel_size,padding="same")
        self.maxPool = MaxPooling2D(pool_size=pool_size)
        self.batch = BatchNormalization()
        
    
    def call(self,input_tensor,training=False):

        x = self.conv(input_tensor)
        x = self.batch(x,training=training)
        x = tf.nn.relu(x)
        #x = self.maxPool(x)
       
        return x
      

In [ ]:
class myResBlock(Layer):

    def __init__(self,channels):
        super(myResBlock,self).__init__()


        self.cnn1 = myCNNBlock(channels[0],3)
        self.cnn2 = myCNNBlock(channels[1],3)
        self.cnn3 = myCNNBlock(channels[2],3)
        
        self.pooling = MaxPooling2D()
        self.identity_mapping = Conv2D(channels[1],3,padding="same")

    def call(self,input_tensor,training=False):
        
        x = self.cnn1(input_tensor,training=training)
        x = self.cnn2(x,training=training)
        
        x = self.cnn3(x + self.identity_mapping(input_tensor) ,training=training)
        x = self.pooling(x)
        
        return x
    

In [ ]:
class ResNet(keras.Model):

    def __init__(self,number_of_classes=10):
        
        super(ResNet,self).__init__()
    
        self.res1 = myResBlock([32,32,64])
        self.res2 = myResBlock([128,128,256])
        self.res3 = myResBlock([128,256,512])

        self.pool = GlobalAveragePooling2D()
        self.classifier = Dense(number_of_classes)
    
    def call(self,input_tensor,training=False):
        #input_tensor = keras.Input(shape=input_tensor)
        x = self.res1(input_tensor,training=training)
        x = self.res2(x,training=training)
        x = self.res3(x,training=training)
        x = self.pool(x)
        x = self.classifier(x)
        return x


    def model(self):
        x = keras.Input(shape=(64,64,3))
        return keras.Model(inputs=[x],outputs=self.call(x))


In [ ]:
model2 = ResNet().model()
from keras.optimizers import Adam
model2.compile(optimizer=Adam(learning_rate = 0.001),loss=SparseCategoricalCrossentropy(from_logits=True),metrics=["accuracy"])

In [ ]:
model2.summary()

In [ ]:
model2.fit(train_ds,epochs=3,verbose=2)